Downloading Zip

In [1]:
!gdown https://drive.google.com/uc?id=1Lbk_pwATorDcUyk9VcGYzomLR4Rhj-lv
zip_path = '/content/Cat_GAN_1.zip' 
!unzip -q Cat_GAN_1.zip 
!rm Cat_GAN_1.zip 

Downloading...
From: https://drive.google.com/uc?id=1Lbk_pwATorDcUyk9VcGYzomLR4Rhj-lv
To: /content/Cat_GAN_1.zip
100% 1.13G/1.13G [00:12<00:00, 93.6MB/s]


Importing Libraries

In [2]:
import os 
import pdb
import cv2
import operator
from PIL import Image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2DTranspose, Dense, LeakyReLU, Input, Reshape
from tensorflow.keras.layers import Conv2D, Flatten, MaxPool2D, BatchNormalization, Activation
import numpy as np
import matplotlib.pyplot as plt
import math
import random

Defining Upscale Factor

In [3]:
upscale_factor = 16
if math.log(upscale_factor, 2) % 1.0 != 0.0:
  print('Upscale factor is not a power of 2: Will be rounded to the nearest power of 2 by a logarithmic scale')
  upscale_factor = int(2**round(math.log(upscale_factor, 2)))
  print('New upscale factor is %d' % (upscale_factor))

Preprocessing Data

In [4]:
os.mkdir("Training") #making the new Training and testing folders
datadir1 = "/content/Cat_Classifier_Images_70-30 /Train/Domino"
datadir2 = "/content/Cat_Classifier_Images_70-30 /Train/Stormy"
datadir3 = "/content/Cat_Classifier_Images_70-30 /Test/Domino"
datadir4 = "/content/Cat_Classifier_Images_70-30 /Test/Stormy"
filelist1 = sorted(os.listdir(datadir1), key = lambda fname: int(fname.split("_")[0][-4:]))
filelist2 = sorted(os.listdir(datadir2), key = lambda fname: int(fname.split("_")[0][-4:]))
filelist3 = sorted(os.listdir(datadir3), key = lambda fname: int(fname.split("_")[0][-4:]))
filelist4 = sorted(os.listdir(datadir4), key = lambda fname: int(fname.split("_")[0][-4:]))
datadirs = [filelist1, filelist2, filelist3, filelist4]
inc = 0
idom = 0
istorm = 0
for filelist in datadirs:
  for fil in filelist:
    if inc == 0:
      path = "/content/Cat_Classifier_Images_70-30 /Train/Domino/" + fil
      idom += 1
      img = cv2.imread(path)
      imgResized = cv2.resize(img, (48*upscale_factor, 64*upscale_factor))
      cv2.imwrite('/content/Training/DominoTR%03i.jpg' %idom, imgResized)
    elif inc == 1:
      path = "/content/Cat_Classifier_Images_70-30 /Train/Stormy/" + fil
      istorm += 1
      img = cv2.imread(path)
      imgResized = cv2.resize(img, (48*upscale_factor, 64*upscale_factor))
      cv2.imwrite('/content/Training/StormyTR%03i.jpg' %istorm, imgResized)
    elif inc == 2:
      path = "/content/Cat_Classifier_Images_70-30 /Test/Domino/" + fil
      img = cv2.imread(path)
      idom += 1
      imgResized = cv2.resize(img, (48*upscale_factor, 64*upscale_factor))
      cv2.imwrite('/content/Training/DominoTR%03i.jpg' %idom, imgResized)
    else:
      path = "/content/Cat_Classifier_Images_70-30 /Test/Stormy/" + fil
      img = cv2.imread(path)
      istorm += 1
      imgResized = cv2.resize(img, (768, 1024))
      cv2.imwrite('/content/Training/StormyTR%03i.jpg' %istorm, imgResized)
  inc += 1

Defining Model

In [9]:
# Variable parameters

hidden_units = 32
kernel_size = 7

# Defining the network

inputs = Input(shape=(64, 48, 3))
result = Conv2DTranspose(hidden_units, kernel_size=kernel_size, padding='same', activation='relu', name='initial_transpose')(inputs)
for i in range(int(math.log(upscale_factor, 2))):
  name = 'upscale_%d' % (i + 1)
  result = Conv2DTranspose(hidden_units, kernel_size=kernel_size, strides=2, padding='same', activation='relu', name=name)(result)
result = Conv2DTranspose(hidden_units, kernel_size=kernel_size, padding='same', activation='relu', name='penultimate')(result)
result = Conv2DTranspose(3, kernel_size=kernel_size, padding='same', activation='sigmoid', name='final_transpose')(result)
catUpscaler = Model(inputs=inputs, outputs=result, name='Cat_Upscaler')
catUpscaler.compile(loss='mae', optimizer='adam')
print(catUpscaler.summary())

Model: "Cat_Upscaler"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 64, 48, 3)]       0         
                                                                 
 initial_transpose (Conv2DTr  (None, 64, 48, 32)       4736      
 anspose)                                                        
                                                                 
 conv2d_transpose (Conv2DTra  (None, 64, 48, 32)       50208     
 nspose)                                                         
                                                                 
 upscale_1 (Conv2DTranspose)  (None, 128, 96, 32)      50208     
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 128, 96, 32)      50208     
 ranspose)                                                       
                                                      

Training Loop

In [ ]:
training_samples = len(os.listdir('/content/Training'))

# Hyperparameters

epochs = 1000
batch_size = 15
gen_interval = 50  # Inrements from which you see the low vs high res images
gen_amount = 4  # Amount of images (must be perfect square)

# Main Loop

high_res = []
low_res = []
show_low_res = []
for episode in range(epochs):
  rand_indexes = np.random.randint(0, training_samples-1, size=batch_size)
  for ind in rand_indexes:
    img = cv2.imread('/content/Training/' + os.listdir('/content/Training')[ind])
    high_res.append(np.array(img) / 256) #Normalizing image
    low_res.append(np.array(cv2.resize(img, (48, 64))) / 256)
  high_res = np.array(high_res)
  low_res = np.array(low_res)
  loss = catUpscaler.train_on_batch(low_res, high_res)
  print ('%d: [loss: %s]' % (episode + 1, round(loss, 5)))
  low_res = []
  high_res = []
  if (episode + 1) % gen_interval == 0:
    indexes = np.random.randint(0, training_samples-1, size=gen_amount)
    for ind in indexes:
      img = cv2.imread('/content/Training/' + os.listdir('/content/Training')[ind])
      show_low_res.append(np.array(cv2.resize(img, (48, 64))) / 256)
    show_low_res = np.array(show_low_res)
    new_images = catUpscaler.predict(show_low_res)
    plt.figure(figsize=(22.2, 22.2))
    num_images = new_images.shape[0]
    image_size = new_images.shape[1]
    rows = int(math.sqrt(gen_amount))
    for i in range(num_images):
      plt.subplot(rows, rows, i + 1)
      plt.imshow(np.array(new_images[i - 1]).reshape((64*upscale_factor, 48*upscale_factor, 3)))
      plt.axis('off')
    plt.show()
    show_low_res = []

Saving Model

In [8]:
catUpscaler.save('Cat_Upscaler.h5')